# DisSysLab Colab Wizard — Track A

A friendly Colab notebook that:
- Runs **token‑free** demos (Uppercase / Reverse / Word Count / Scale Numbers).
- **Optionally** unlocks GPT features **only if** you enter an OpenAI API key.
- Lets you **install DisSysLab from an uploaded wheel** so your repo can stay private during development.

> Tip: Run cells top‑to‑bottom the first time.


## 1) Environment setup (safe to run multiple times)

- Installs minimal deps for widgets and OpenAI.
- If you're **not** using OpenAI features, this still runs fine.


In [ ]:
# If you're on Colab, this cell is safe to run multiple times.
# It installs widgets and the OpenAI SDK for the optional GPT features.
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Core deps
if IN_COLAB:
    # Widgets for the UI
    !pip -q install ipywidgets
    # Latest OpenAI SDK (only needed if you plan to use GPT features)
    !pip -q install --upgrade openai>=1.40.0
else:
    print("Note: You're not on Colab. The notebook still works in Jupyter, but upload UI may differ.")


## 2) Upload and install your DisSysLab wheel (optional)

- Click the **Run** button and choose your `.whl` file.
- The cell will `pip install` the first wheel it sees.
- After install, the next cell tries a **best-guess import** based on the wheel name (you can edit as needed).


In [ ]:
# Upload your wheel file (e.g., dissyslab-0.1.0-py3-none-any.whl)
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    wheel_files = [name for name in uploaded.keys() if name.endswith(".whl")]
    if wheel_files:
        wheel = wheel_files[0]
        print(f"Installing wheel: {wheel}")
        !pip -q install "{wheel}"
    else:
        print("No .whl uploaded; skipping install.")
except Exception as e:
    print("Upload UI not available (likely not Colab). If local, place your .whl next to the notebook and pip install it.")


### (Optional) Import DisSysLab

This cell tries to import a module name inferred from your wheel filename. If it fails, just **edit the import** to match your package name.


In [ ]:
import sys, glob, os, re, importlib

def guess_module_from_wheel_name(wheel_name: str) -> str | None:
    # Guess top-level module from a typical wheel filename: name-version-...whl
    m = re.match(r"([A-Za-z0-9_\-]+)-\d", os.path.basename(wheel_name))
    if not m:
        return None
    # Normalize common python package naming to import-style
    name = m.group(1).replace("-", "_").lower()
    return name

guessed = None
for whl in glob.glob("*.whl"):
    guessed = guess_module_from_wheel_name(whl)
    if guessed:
        break

if guessed:
    print(f"Trying to import '{guessed}' from wheel…")
    try:
        mod = importlib.import_module(guessed)
        print(f"Success: imported '{guessed}'")
    except Exception as e:
        print(f"Couldn't import '{guessed}'. Edit the line below to import your real package name.\nError: {e}")
else:
    print("No local wheel found to guess import name from. Manually import your package below if already installed.")

# Example (edit to match your package):
# from dissyslab import __version__ as dsl_version
# print("DisSysLab version:", dsl_version)


## 3) Token‑free demos

These utilities **do not** require any API key. Use them to verify the notebook UI quickly.


In [ ]:
import re
import ipywidgets as w
from IPython.display import display, clear_output

def do_uppercase(text: str) -> str:
    return text.upper()

def do_reverse(text: str) -> str:
    return text[::-1]

def do_word_count(text: str) -> str:
    words = re.findall(r"\b\w+\b", text)
    return f"Word count: {len(words)}"

def do_scale_numbers(text: str, factor: float) -> str:
    def repl(m):
        num = float(m.group(0))
        return str(num * factor)
    return re.sub(r"(?<![\w.])(?:\d+(?:\.\d+)?)", repl, text)

# Widgets
input_text = w.Textarea(placeholder="Type some text here…", layout=w.Layout(width="100%", height="120px"))
op = w.Dropdown(options=["Uppercase", "Reverse", "Word Count", "Scale Numbers"], value="Uppercase", description="Op")
factor = w.FloatText(value=2.0, description="Factor")
run_btn = w.Button(description="Run", button_style="primary")
out = w.Output()

def on_run_clicked(_):
    with out:
        clear_output(wait=True)
        t = input_text.value or ""
        choice = op.value
        if choice == "Uppercase":
            print(do_uppercase(t))
        elif choice == "Reverse":
            print(do_reverse(t))
        elif choice == "Word Count":
            print(do_word_count(t))
        elif choice == "Scale Numbers":
            try:
                print(do_scale_numbers(t, float(factor.value)))
            except Exception as e:
                print("Scaling error:", e)

run_btn.on_click(on_run_clicked)

# Layout
scale_box = w.HBox([factor])
scale_box.layout.display = "none"
def toggle_factor(*_):
    scale_box.layout.display = "" if op.value == "Scale Numbers" else "none"
op.observe(toggle_factor, names="value")
toggle_factor()

display(w.VBox([input_text, w.HBox([op, run_btn]), scale_box, out]))


## 4) Optional: OpenAI GPT features

Only needed if you want to use GPT. If you leave the key blank, this section simply won’t run anything.

**Notes:**  
- Default model is set to `gpt-4o-mini` but you can change it.  
- If you ever see **“model not found”**, double‑check the model name and that your key has access.


In [ ]:
import os, json, traceback
import ipywidgets as w
from IPython.display import display, clear_output

# UI
api_key = w.Password(description="API Key", placeholder="sk-...")
model = w.Text(value="gpt-4o-mini", description="Model")
system_prompt = w.Textarea(value="You are a helpful assistant.", description="System", layout=w.Layout(width="100%", height="80px"))
user_prompt = w.Textarea(value="Say hello and summarize this prompt.", description="User", layout=w.Layout(width="100%", height="100px"))
run_btn = w.Button(description="Run GPT", button_style="success")
out = w.Output()

def run_gpt(_):
    with out:
        clear_output(wait=True)
        key = api_key.value.strip()
        if not key:
            print("No API key provided. Enter an API key to enable GPT features.")
            return
        try:
            os.environ["OPENAI_API_KEY"] = key
            from openai import OpenAI
            client = OpenAI()
            resp = client.responses.create(
                model=model.value.strip(),
                input=[
                    {"role": "system", "content": system_prompt.value},
                    {"role": "user", "content": user_prompt.value},
                ]
            )
            # Support both object shapes
            try:
                text = resp.output[0].content[0].text
            except Exception:
                # Fallback for older/newer variants
                text = getattr(resp, "text", None) or json.dumps(resp.model_dump(), indent=2)[:4000]
            print(text)
        except Exception as e:
            print("Error while calling OpenAI:")
            traceback.print_exc(limit=1)

display(w.VBox([w.HBox([api_key, model]), system_prompt, user_prompt, run_btn, out]))
run_btn.on_click(run_gpt)


---

### Troubleshooting
- **“model not found”**: The model name may have changed or your key lacks access. Try `gpt-4o-mini`, `gpt-4o`, or check your account’s available models.
- If a previous download link stopped working, just re‑upload or re‑run this notebook. Copies can go stale after model or runtime changes.
